In [ ]:
import sqlite3

In [ ]:
import spacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")

In [ ]:
# Add the project root directory to the Python path
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [ ]:
from evaluation_pipeline.evaluation import run_traditional_eval

#### Fetch top frecent items

In [ ]:
row_limit = 10000
GENERATE_TOPIC = False

In [ ]:
firefox_conn = sqlite3.connect("../data/places.sqlite")  
firefox_cursor = firefox_conn.cursor()

input_data = firefox_cursor.execute(f"""
WITH TOP_FRECENT_PLACES AS
(SELECT p.url, p.title, p.description, p.id AS place_id, p.frecency, p.origin_id, p.url_hash
FROM moz_places p
WHERE p.title NOTNULL
AND url not like '%google.com/search?%'
ORDER BY frecency DESC
LIMIT {row_limit}
) 

, TOP_PLACES_INFO AS
(select * from TOP_FRECENT_PLACES
UNION

SELECT p.url, p.title, p.description, p.id AS place_id, p.frecency, p.origin_id, p.url_hash
FROM moz_places p
WHERE p.id in (select distinct(place_id) from moz_inputhistory)
)
, KEYWORDS_INFO AS
(SELECT 
    ih.place_id, 
    json_group_array(
        json_object(
            'keyword', ih.input,
            'use_count', ih.use_count
        )
    ) AS keyword_data
FROM 
    moz_inputhistory ih
WHERE ih.input != ''
GROUP BY 
    ih.place_id
ORDER BY 
    ih.use_count DESC
)

, DOMAIN_INFO AS
(SELECT 
    id AS origin_id, 
    host, 
    CAST(frecency AS REAL) / (SELECT SUM(frecency) * 1.0 FROM moz_origins WHERE frecency IS NOT NULL) AS domain_frecency
FROM 
    moz_origins
WHERE 
    frecency IS NOT NULL
)

SELECT p.*, kw.keyword_data, d.host, d.domain_frecency 
FROM TOP_PLACES_INFO p
LEFT JOIN KEYWORDS_INFO kw
  ON p.place_id = kw.place_id
LEFT JOIN DOMAIN_INFO d
  ON p.origin_id = d.origin_id
ORDER BY p.frecency DESC

""").fetchall()

In [ ]:
import pandas as pd

input_data_df = pd.DataFrame(input_data, 
             columns=['url', 'title', 'description', 'place_id', 'frecency', 'origin_id', 'url_hash', 'keyword_data', 'host', 'domain_frecency'])

In [ ]:
def extract_additional_path_info(row):
    url = row['url']
    host = row['host']
    path = url.replace(f"https://{host}", "").replace(f"http://{host}", "")
    path = path.strip("/")
    path = path.replace(".html", "").replace(".htm", "")
    path_info = path.split("/")
    return path_info

def extract_tags_batch(df):
    # Combine title and description into a single text column
    texts = (df['title'].fillna('') + " " + df['description'].fillna('')).str.strip()
    
    # Process texts in batch using spaCy's pipe
    docs = nlp.pipe(texts, disable=["ner"])  # Disable unnecessary components for speed

    # Extract tags for each document
    tags_list = []
    for doc in docs:
        tags = set()
        
        # Extract noun chunks and proper nouns
        # for chunk in doc.noun_chunks:
        #     tags.add(chunk.text.strip().lower())
        for token in doc:
            if token.pos_ in ["ADJ", "PROPN", "NOUN"] and not token.is_stop:
                tags.add(token.text.strip().lower())
        
        tags_list.append(list(tags))  # Append the tags for this document
    
    return tags_list

In [ ]:
input_data_df['path_info'] = input_data_df.apply(lambda row: extract_additional_path_info(row), axis=1)
input_data_df['tags'] = extract_tags_batch(input_data_df)


In [ ]:
input_data_df

In [ ]:
input_data_df['domain_frecency'].describe()

In [ ]:
input_data_df['tags'].values[:20]

In [ ]:
input_data_df.sample(20).T

In [ ]:
input_data_df.sample(20)['title'].values

#### Extract the topics

In [ ]:
from gliner import GLiNER

gliner_model = GLiNER.from_pretrained("urchade/gliner_largev2")

labels = [ "Arts & Entertainment",
              "Business and Consumer Services",
              "Community and Society",
              "Computers Electronics and Technology",
              "Ecommerce & Shopping",
              "Finance",
              "Food and Drink",
              "Gambling",
              "Games",
              "Health",
              "Heavy Industry and Engineering",
              "Hobbies and Leisure",
              "Home and Garden",
              "Jobs and Career",
              "Law and Government",
              "Lifestyle",
              "News & Media Publishers",
              "Pets and Animals",
              "Reference Materials",
              "Science and Education",
              "Sports",
              "Travel and Tourism",
              "Vehicles",
              "Adult"
             ]

In [ ]:
texts = (input_data_df['title'].fillna('') + " " + input_data_df['description'].fillna('')).values.tolist()

In [ ]:
len(texts)

In [ ]:
texts[:5]

In [ ]:
## Very first time set this to True and then switch to False and read from saved file
# GENERATE_TOPIC = False

if GENERATE_TOPIC:
    topics = []
    for text in tqdm(texts):
        entities = gliner_model.predict_entities(text, labels, threshold=0.3)
        themes = list({entity["label"] for entity in entities})
        topics.append(themes)
    input_data_df['topics'] = topics
    input_data_df.to_parquet("../data/input_data_df.parquet", index=False)
else:
    input_data_df_bkp = pd.read_parquet("../data/input_data_df.parquet")
    topics_lkp = input_data_df_bkp.set_index('url_hash')['topics'].to_dict()
    input_data_df['topics'] = input_data_df['url_hash'].map(topics_lkp)

In [ ]:
len(input_data_df)

In [ ]:
input_data_df

In [ ]:
from collections import Counter

tags_counter = Counter()
tags_counter.update([tag for tags in input_data_df['tags'].values.tolist() for tag in tags if tag.isalnum()])

In [ ]:
len(tags_counter)

In [ ]:
tags_counter.most_common(10)

In [ ]:
path_info_counter = Counter()
path_info_counter.update(
    [path_i for path_info in input_data_df['path_info'].values.tolist() for path_i in path_info if len(path_i) > 2 and path_i.isalpha()]
)
print(len(path_info_counter))
path_info_counter.most_common(30)

In [ ]:
import json

def extract_keywords_adhoc(json_str):
    try:
        # Parse the string as JSON
        data = json.loads(json_str)
        # Extract the "keyword" field from each dictionary
        return [item["keyword"] for item in data]
    except (json.JSONDecodeError, TypeError):
        # Handle invalid JSON or None
        return []

keywords_list = input_data_df['keyword_data'].apply(extract_keywords_adhoc).values.tolist()
kws_counter = Counter()
kws_counter.update([kw for kws in keywords_list for kw in kws])

In [ ]:
len(kws_counter)

In [ ]:
input_data_df['keyword_data'][(~input_data_df['keyword_data'].isna())]

In [ ]:
def generate_entity_rltn_score(src_entity, src_entity_type, relation, tgt_entity, score):
    return (src_entity, src_entity_type, relation, tgt_entity, score)

def extract_keyword_entities_rltn_score(df, entity_name, entity_type, relation, tgt_entity_name, score_col=None):
    sel_df = df.loc[~df[entity_name].isna(), [entity_name, tgt_entity_name]].reset_index(drop=True)
    for ers_info, tgt_val in zip(sel_df[entity_name].apply(json.loads), sel_df[tgt_entity_name]):
        for ers in ers_info:
            for key, val in ers.items():
                # print(key, val, tgt_val)
                if key == entity_type:
                    src_entity = val
                if score_col and key == score_col:
                    score = 1+val
                else:
                    score = None
            yield generate_entity_rltn_score(src_entity, entity_type, relation, tgt_val, score)
    
    
def extract_domain_entities_rltn_score(df, entity_name, relation, tgt_entity_name, score_col=None):
    sel_df = df.loc[~df[entity_name].isna(), [entity_name, tgt_entity_name, score_col]].reset_index(drop=True)
    for idx, row in sel_df.iterrows():
        yield generate_entity_rltn_score(row[entity_name], entity_name, relation, row[tgt_entity_name], row[score_col])

def extract_path_info_entities_rltn_score(df, entity_name, relation, tgt_entity_name, score_col=None):
    sel_df = df.loc[~df[entity_name].isna(), [entity_name, tgt_entity_name]].reset_index(drop=True)
    for idx, row in sel_df.iterrows():
        for entity_val in row[entity_name]:
            if len(entity_val) > 2 and entity_val.isalpha():
                yield generate_entity_rltn_score(entity_val, entity_name, relation, row[tgt_entity_name], score_col) 

def extract_tags_entities_rltn_score(df, entity_name, relation, tgt_entity_name, score_col=None):
    sel_df = df.loc[~df[entity_name].isna(), [entity_name, tgt_entity_name]].reset_index(drop=True)
    for idx, row in sel_df.iterrows():
        for entity_val in row[entity_name]:
            if len(entity_val) > 2 and entity_val.isalnum():
                yield generate_entity_rltn_score(entity_val, 'tag', relation, row[tgt_entity_name], score_col) 

def extract_topics_entities_rltn_score(df, entity_name, relation, tgt_entity_name, score_col=None):
    sel_df = df.loc[~df[entity_name].isna(), [entity_name, tgt_entity_name]].reset_index(drop=True)
    for idx, row in sel_df.iterrows():
        for entity_val in row[entity_name]:
            if len(entity_val) > 1:
                yield generate_entity_rltn_score(entity_val, 'topic', relation, row[tgt_entity_name], score_col) 

In [ ]:
# print(next(generate_entity_rltn_score('cloud', 'keyword', 'refers_to', 'place_id1', 0.391895954969)))
# print(next(extract_entities_rltn_score(input_data_df, 'keyword_data', 'keyword', 'refers_to', 'place_id', 'use_count')))
keyword_ers = [ers for ers in (extract_keyword_entities_rltn_score(input_data_df, 'keyword_data', 'keyword', 'refers_to', 'url_hash', 'use_count'))]
print(len(keyword_ers))
keyword_ers[:5]

In [ ]:
domain_ers = [ers for ers in extract_domain_entities_rltn_score(input_data_df, 'host', 'contains', 'url_hash', 'domain_frecency')]
print(len(domain_ers))
domain_ers[:5]

In [ ]:
path_info_ers = [ers for ers in extract_path_info_entities_rltn_score(input_data_df, 'path_info', 'parses_to', 'url_hash', 1.0)]
print(len(path_info_ers))
path_info_ers[:5]

In [ ]:
tags_ers = [ers for ers in extract_tags_entities_rltn_score(input_data_df, 'tags', 'tagged_has', 'url_hash', 1.0)]
print(len(tags_ers))
tags_ers[:5]

In [ ]:
topics_ers = [ers for ers in extract_topics_entities_rltn_score(input_data_df, 'topics', 'belongs_to', 'url_hash', 1.0)]
print(len(topics_ers))
topics_ers[:5]

#### Combining all entities and relationships

In [ ]:
len(keyword_ers) + len(domain_ers ) + len(path_info_ers) + len(tags_ers) + len(topics_ers)

In [ ]:
ers_df = pd.DataFrame(keyword_ers + domain_ers + path_info_ers + tags_ers + topics_ers,
            columns=['entity', 'entity_type', 'relation', 'url_hash', 'score'])
# + len(domain_ers ) + len(path_info_ers) + len(tags_ers) + len(topics_ers)

In [ ]:
ers_df

In [ ]:
# Create a new SQLite database
db_path = "../data/ml_kg.db"
conn = sqlite3.connect(db_path)

# Create the ml_kg_info table
conn.execute("""
CREATE TABLE IF NOT EXISTS ml_kg_info (
    entity TEXT NOT NULL,
    entity_type TEXT NOT NULL,
    relation TEXT NOT NULL,
    url_hash INTEGER NOT NULL,
    score REAL NOT NULL
);
""")

# Insert data from DataFrame into the table
ers_df.to_sql("ml_kg_info", conn, if_exists="append", index=False)

# Attach the moz_places table from places.sqlite
places_db_path = "../data/places.sqlite"
conn.execute(f"ATTACH DATABASE '{places_db_path}' AS places_db;")

In [ ]:
def extract_tags_for_queries(queries):
    texts = queries[::]
    
    docs = nlp.pipe(texts, disable=["ner"])

    tags_list = []
    for doc in docs:
        tags = set()
        
        for token in doc:
            # print(token.pos_)
            if token.pos_ in ["ADJ", "PROPN", "NOUN"] and not token.is_stop:
                tags.add(token.text.strip().lower())
        
        tags_list.append(list(tags))  # Append the tags for this document
    
    return tags_list

def infer_topics(queries, pbar=True):
    topics = []
    if pbar:
        for query in tqdm(queries):
            entities = gliner_model.predict_entities(query, labels, threshold=0.3)
            themes = list({entity["label"] for entity in entities})
            topics.append(themes)
    else:
        for query in queries:
            entities = gliner_model.predict_entities(query, labels, threshold=0.3)
            themes = list({entity["label"] for entity in entities})
            topics.append(themes)
    return topics
    
def fetch_entity_relations_with_keywords(conn, search_keyword, search_tags, search_topics):
    # Convert the list of search keywords into a string suitable for SQL
    if not search_keyword:
        raise ValueError("search_keywords list cannot be empty.")

    keyword_placeholder = f"'{search_keyword}'"
    # print("keyword_placeholder = ", keyword_placeholder)
    
    tag_placeholder = ', '.join(f"'{tag}'" for tag in search_tags)
    # print("tag_placeholder = ", tag_placeholder)

    topic_placeholder = ', '.join(f"'{topic}'" for topic in search_topics)
    # print("topic_placeholder = ", topic_placeholder)

    # Define the query with the dynamic IN clause
    query = f"""
    WITH entity_relations_info AS (
        SELECT 
            m.entity,
            m.entity_type,
            m.relation,
            m.url_hash,
            m.score,
            p.url,
            p.title,
            p.frecency
        FROM 
            ml_kg_info m
        JOIN 
            places_db.moz_places p
        ON 
            m.url_hash = p.url_hash
        WHERE
            (m.entity IN ({keyword_placeholder}) AND
            m.entity_type = 'keyword') OR
            (m.entity IN ({tag_placeholder})  AND
             m.entity_type = 'tag') OR
            (m.entity IN ({topic_placeholder})  AND
             m.entity_type = 'topic')
        ORDER BY 
            m.score DESC
    )

    SELECT 
        url_hash, 
        url, 
        title,
        SUM(score) AS total_score
    FROM 
        entity_relations_info
    GROUP BY 
        url_hash, url, title
    ORDER BY 
        total_score DESC;
    
    """
    
    

    results = pd.read_sql_query(query, conn)
    return results


# search_query = "kanba"
search_query = "healthy food and education"
# search_keywords = search_query.split(" ")
search_tags = extract_tags_for_queries([search_query])[0]
search_topics = infer_topics([search_query])[0]


results = fetch_entity_relations_with_keywords(conn, search_query, search_tags, search_topics)


In [ ]:
results.head(10).T

#### Validation

In [ ]:

def fetch_ground_truths():
    val_cursor = firefox_conn.cursor()

    val_data = val_cursor.execute(
        """
        SELECT ih.input AS keyword,
               p.url_hash,
               ih.use_count,
               p.url
          FROM moz_inputhistory ih
          JOIN moz_places p
            ON ih.place_id = p.id
          WHERE input != ''
          ORDER BY keyword, use_count DESC
        """
    ).fetchall()
    return val_data
    

In [ ]:
val_data = fetch_ground_truths()
actuals_df = pd.DataFrame(val_data, columns=['keyword', 'url_hash', 'use_count', 'url'])
to_be_predicted_queries = actuals_df.groupby('keyword')['url_hash'].agg(list).reset_index()
print(len(to_be_predicted_queries))

In [ ]:
def perform_traditional_evals(to_be_predicted_queries, use_tags=True, use_topics=True):
    eval_rows = []
    for idx, row in to_be_predicted_queries.iterrows():
        if (idx+1) % 50 == 0:
            print(f" {idx+1} queries evaluated")
        search_query = search_keyword = row['keyword']
        # print(f"search_keyword = {search_keyword}")
        relevant_docs = row['url_hash']
        
        search_tags = extract_tags_for_queries([search_query])[0] if use_tags else []
        search_topics = infer_topics([search_query], pbar=False)[0] if use_topics else []
        # print(f"search_tags = {search_tags}")
        # print(f"search_topics = {search_topics}")
        results = fetch_entity_relations_with_keywords(conn, search_keyword, search_tags, search_topics).head(2)
        retrieved_docs = []
        if len(results) > 0:
            retrieved_docs = results['url_hash'].values.tolist()
            eval_row = run_traditional_eval(idx, search_keyword, relevant_docs, retrieved_docs, retrieved_distances=None, k=2)
            eval_rows.append(eval_row)
    return pd.DataFrame(eval_rows)


#### Use keywords + tags + topics

In [ ]:
eval_df = perform_traditional_evals(to_be_predicted_queries)
# print(eval_df)
eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean()


# # keywords + tags + topics 
# precision@2          0.539931
# recall@2             0.964699
# ndcg@2               0.590278
# reciprocal_rank      0.968750
# average_precision    0.470486

#### Use only keywords 

In [ ]:
eval_df = perform_traditional_evals(to_be_predicted_queries, use_tags=False, use_topics=False)
# print(eval_df)
eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean()

#### Use  keywords + Tags and no topics

In [ ]:
eval_df = perform_traditional_evals(to_be_predicted_queries, use_tags=True, use_topics=False)
# print(eval_df)
eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean()

#### Use  keywords + topics and no tags

In [ ]:
eval_df = perform_traditional_evals(to_be_predicted_queries, use_tags=False, use_topics=True)
print(len(eval_df))
eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean()

In [ ]:
# firefox_conn.close()

In [ ]:
to_be_predicted_queries

In [ ]:
golden_queries = pd.read_csv("../data/chidam_golden_query.csv", usecols=['search_query', 'url'])
print(len(golden_queries))
golden_queries
# set(golden_queries['search_query'].tolist()).intersection(set(to_be_predicted_queries['keyword'].values.tolist()))

In [ ]:
set(golden_queries['search_query'].tolist()).intersection(set(to_be_predicted_queries['keyword'].values.tolist()))

In [ ]:
set(golden_queries['search_query'].tolist())

In [ ]:
set(to_be_predicted_queries['keyword'].values.tolist())

In [ ]:
def get_url_hash_batch(golden_queries, firefox_conn):
    # Convert URLs into a tuple for the IN clause
    urls = tuple(golden_queries['url'].tolist())

    # Query all URL hashes in one go
    query = f"""
    SELECT url, url_hash
    FROM moz_places
    WHERE url IN ({','.join(['?'] * len(urls))})
    """
    
    # Execute the query and fetch results
    cursor = firefox_conn.cursor()
    results = cursor.execute(query, urls).fetchall()

    # Convert results into a DataFrame
    return pd.DataFrame(results, columns=["url", "url_hash"])

# Example usage
url_hashes = get_url_hash_batch(golden_queries, firefox_conn)
golden_queries_updated = golden_queries.merge(url_hashes, on='url', how='inner')
golden_queries_updated = golden_queries_updated.groupby('search_query')['url_hash'].agg(list).reset_index()\
                                               .rename(columns={'search_query': 'keyword'})
print(len(golden_queries_updated))

In [ ]:
golden_queries_updated

In [ ]:
## keywords + tags + topics
golden_eval_df = perform_traditional_evals(golden_queries_updated)
print(golden_eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean())
print(len(golden_eval_df))
golden_eval_df

In [ ]:
## only keywords
golden_eval_df = perform_traditional_evals(golden_queries_updated, use_tags=False, use_topics=False)
print(golden_eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean())
print(len(golden_eval_df))
golden_eval_df

In [ ]:
## keywords + tags and no topics
golden_eval_df = perform_traditional_evals(golden_queries_updated, use_tags=True, use_topics=False)
print(golden_eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean())
print(len(golden_eval_df))
golden_eval_df

In [ ]:
## keywords + topics and no tags
golden_eval_df = perform_traditional_evals(golden_queries_updated, use_tags=False, use_topics=True)
print(golden_eval_df[['precision@2','recall@2','ndcg@2','reciprocal_rank','average_precision']].mean())
print(len(golden_eval_df))
golden_eval_df